<a href="https://colab.research.google.com/github/Lipeya/EMNIST_classify/blob/main/CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#google drive 연동
from google.colab import drive
drive.mount('/content/mnt')

Mounted at /content/mnt


In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D,Conv1D, MaxPooling2D, MaxPooling1D, Dropout, Flatten
from keras import losses
from keras import optimizers
from keras.utils import np_utils
import numpy as np
import pandas as pd
import csv
import math

class_array = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'd', 'e', 'f', 'g', 'h', 'n', 'q', 'r','t']
#class_array = ['a', 'b','c', 'd', 'e', 'f', 'g', 'h', 'i','j','k','l','m','n','o','p', 'q', 'r','s', 't','u','v','w','x','y','z']
class_dict = {'0':0, '1':1, '2':2, '3':3, '4':4, '5':5, '6':6, '7':7, '8':8, '9':9, 'A':10, 'B':11, 'C':12, 'D':13, 'E':14, 'F':15, 'G':16, 'H':17, 'I':18, 'J':19, 'K':20, 'L':21, 'M':22, 'N':23, 'O':24, 'P':25, 'Q':26, 'R':27, 'S':28, 'T':29, 'U':30, 'V':31, 'W':32, 'X':33, 'Y':34, 'Z':35, 'a':36, 'b':37, 'd':38, 'e':39, 'f':40, 'g':41, 'h':42, 'n':43, 'q':44, 'r':45,'t':46}
class_size =len(class_array)
print(class_size)

47


In [ ]:
#EMNIST test파일 가져오기

train_data_list = pd.read_csv("/content/mnt/MyDrive/project/emnist-balanced-train.csv") #,chunksize = 100000)
test_data_list = pd.read_csv("/content/mnt/MyDrive/project/emnist-balanced-test.csv")
#predict_data_list = pd.read_csv("/content/mnt/MyDrive/project/predict_char.csv")
predict_train = pd.read_csv("/content/mnt/MyDrive/project/encodedbalancetrain.csv")
predict_test = pd.read_csv("/content/mnt/MyDrive/project/encodedbalancetest.csv")

#파일을 적절하게 수정
#td = list(test_data_list)


# test_data_class = test_data_list.iloc[:,0]

# test_data = test_data_list.iloc[:,1:]
# display(test_data_class)
# display(test_data)



In [ ]:
pdc = np.array(predict_train.iloc[:,1:])
pdctest = np.array(predict_test.iloc[:,1:])

In [ ]:
display(pdc)

array([[1.20353192e-07, 2.48738355e-03, 2.77083200e-05, ...,
        2.18570378e-04, 1.37010366e-02, 1.52825058e-04],
       [3.54118697e-07, 3.58922847e-09, 1.59197029e-08, ...,
        1.80840783e-04, 3.33393784e-03, 3.19025712e-03],
       [6.77668504e-06, 3.22239856e-07, 9.52586959e-07, ...,
        1.21038454e-03, 4.52147302e-04, 9.95731563e-04],
       ...,
       [9.70034244e-15, 2.25028085e-13, 1.51046452e-13, ...,
        1.09610301e-06, 5.91361016e-10, 2.88624306e-05],
       [6.30219832e-09, 1.19965726e-08, 4.80549822e-10, ...,
        2.09863429e-04, 3.45348008e-03, 2.91882432e-03],
       [8.08449090e-03, 6.47734525e-03, 8.29083845e-03, ...,
        7.52150081e-05, 5.75112528e-04, 1.57463233e-04]])

In [ ]:
# train_data =  np.array(td[0].iloc[:,1:])
# #train_data = train_data.reshape(train_data.shape[0],28,28,1)
# train_data_class = np.array(td[0].iloc[:,0])
# train_data_class = np_utils.to_categorical(train_data_class, class_size)
# display(train_data_class)

train_data =  np.array(train_data_list.iloc[:,1:])
train_data = train_data.reshape(train_data.shape[0],28,28,1)
train_data_class = np.array(train_data_list.iloc[:,0])
train_data_class = train_data_class - 1
origin_tdc = train_data_class
train_data_class = np_utils.to_categorical(train_data_class ,class_size)



# 모델


In [ ]:
#LSTM 모델

import tensorflow as tf
from tensorflow.keras.layers import Embedding, Input, TimeDistributed, Dropout, concatenate, Bidirectional, LSTM, Conv2D, Dense, MaxPooling2D, Flatten
from tensorflow.keras import Model
from tensorflow.keras.initializers import RandomUniform
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model



embedding_dim = 128
char_embedding_dim = 64
dropout_ratio = 0.5
hidden_units = 256
num_filters = 30
kernel_size = 3

#다음 문자 예측
char_inputs = Input(shape=(36,),dtype='float', name='char_predicts_input')
predict = Dense(2*class_size, activation = 'relu')(char_inputs)
predict = Dropout(0.2)(predict)
#CNN
cnn_inputs = Input(shape =(28,28,1), dtype = 'float', name ='CNN_input')
cnn_layers = Conv2D(filters=32, kernel_size = (5,5), strides = (1,1), padding='same', activation = 'relu')(cnn_inputs)
cnn_layers = MaxPooling2D(pool_size=(2,2), strides=(2,2))(cnn_layers)
cnn_layers = Conv2D(filters=64, kernel_size=(3, 3), strides=(1, 1), activation='relu', padding='same')(cnn_layers)
cnn_layers = MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(cnn_layers)
cnn_layers = Flatten()(cnn_layers)
cnn_layers = Dropout(0.3)(cnn_layers)
gap = Dense(2*class_size, activation = 'relu')(cnn_layers)
gap = Dropout(0.2)(gap)


# CNN결과와 다음 문자 예측 결과를 연결
output = concatenate([gap, predict])
output = Dense(2*class_size, activation = 'relu')(output)
output = Dropout(0.2)(output)
#Dense로 합산
output = Dense(class_size, activation = 'softmax')(output)
#model = Model(inputs= [cnn_inputs], outputs = [output])
model = Model(inputs=[cnn_inputs, char_inputs], outputs=[output])
model.compile(loss='categorical_crossentropy', optimizer='nadam',  metrics=['acc'])

In [ ]:
#LSTM X 모델
model = Sequential()
model.add(Conv2D(filters=32, kernel_size=(5, 5), strides=(1, 1), padding='same', activation='relu', input_shape=(28, 28, 1)))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Conv2D(filters=64, kernel_size=(3, 3), strides=(1, 1), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.3))
model.add(Flatten())
model.add(Dense(units=500, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(units=class_size, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.summary()

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 CNN_input (InputLayer)         [(None, 28, 28, 1)]  0           []                               
                                                                                                  
 conv2d_10 (Conv2D)             (None, 28, 28, 32)   832         ['CNN_input[0][0]']              
                                                                                                  
 max_pooling2d_10 (MaxPooling2D  (None, 14, 14, 32)  0           ['conv2d_10[0][0]']              
 )                                                                                                
                                                                                                  
 conv2d_11 (Conv2D)             (None, 14, 14, 64)   18496       ['max_pooling2d_10[0][0]'] 

In [ ]:
#CNN 학습
history = model.fit(x=train_data,
                    y=train_data_class,
                    batch_size=128,
                    epochs=5,
                    verbose=1,
                    validation_split = 0.2)

Epoch 1/5
705/705 [==============================] - 17s 7ms/step - loss: 1.3513 - accuracy: 0.6923 - val_loss: 0.4566 - val_accuracy: 0.8412
Epoch 2/5
705/705 [==============================] - 6s 9ms/step - loss: 0.5317 - accuracy: 0.8217 - val_loss: 0.3915 - val_accuracy: 0.8582
Epoch 3/5
705/705 [==============================] - 5s 7ms/step - loss: 0.4560 - accuracy: 0.8435 - val_loss: 0.3813 - val_accuracy: 0.8651
Epoch 4/5
705/705 [==============================] - 5s 7ms/step - loss: 0.4183 - accuracy: 0.8533 - val_loss: 0.3597 - val_accuracy: 0.8679
Epoch 5/5
705/705 [==============================] - 5s 7ms/step - loss: 0.3870 - accuracy: 0.8617 - val_loss: 0.3596 - val_accuracy: 0.8647


In [ ]:
#CNN+LSTM 학습
history = model.fit(x=[train_data,pdc],
                    y=train_data_class,
                    batch_size=128,
                    epochs=5,
                    verbose=1,
                    validation_split = 0.2)

Epoch 1/5
705/705 [==============================] - 7s 8ms/step - loss: 1.6134 - acc: 0.5812 - val_loss: 0.3055 - val_acc: 0.9043
Epoch 2/5
705/705 [==============================] - 5s 8ms/step - loss: 0.4924 - acc: 0.8490 - val_loss: 0.2366 - val_acc: 0.9192
Epoch 3/5
705/705 [==============================] - 5s 8ms/step - loss: 0.3718 - acc: 0.8846 - val_loss: 0.2056 - val_acc: 0.9307
Epoch 4/5
705/705 [==============================] - 5s 8ms/step - loss: 0.3249 - acc: 0.8983 - val_loss: 0.1903 - val_acc: 0.9353
Epoch 5/5
705/705 [==============================] - 5s 8ms/step - loss: 0.2949 - acc: 0.9068 - val_loss: 0.1845 - val_acc: 0.9380


In [ ]:
test_data =  np.array(test_data_list.iloc[:,1:])
test_data = test_data.reshape(test_data.shape[0],28,28,1)
test_data_class = np.array(test_data_list.iloc[:,0]) -1
origin_test = test_data_class
test_data_class = np_utils.to_categorical(test_data_class, class_size)

print(test_data_class)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [ ]:
#LSTM 학습 확인
score = model.evaluate(x=[test_data, pdctest],y= test_data_class, verbose=0)
print("loss : " + str(score[0]))
print("accuracy : " + str(score[1]))

loss : 0.20243506133556366
accuracy : 0.9377626180648804


In [ ]:
#LSTM X 학습 확인
score = model.evaluate(x=test_data,y= test_data_class, verbose=0)
print("loss : " + str(score[0]))
print("accuracy : " + str(score[1]))

loss : 0.3748140037059784
accuracy : 0.8674397468566895
